In [1]:
# Importing the Required Packages
from pyspark import SparkContext,SQLContext
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import lower,col,upper

In [2]:
# Reading the FIles using spark.read utilities
# File uploaded to /FileStore/tables/PersonalData.txt
# File uploaded to /FileStore/tables/TravelData.txt

In [3]:
tr_data_df.count()

Out[ 32 ]: 891

In [4]:
ps_data_df=spark.read.csv('/FileStore/tables/PersonalData.txt',header=True,sep='\t',inferSchema=True)
tr_data_df=spark.read.format('csv').option('header',True).option('sep','\t').option('inferSchema',True).load('/FileStore/tables/TravelData.txt')

In [5]:
ps_data_df.show()

+-----------+--------+--------------------+------+
PassengerId|Survived| Name| Sex|
+-----------+--------+--------------------+------+
 1| 0|Braund, Mr. Owen ...| male|
 2| 1|Cumings, Mrs. Joh...|female|
 3| 1|Heikkinen, Miss. ...|female|
 4| 1|Futrelle, Mrs. Ja...|female|
 5| 0|Allen, Mr. Willia...| male|
 6| 0| Moran, Mr. James| male|
 7| 0|McCarthy, Mr. Tim...| male|
 8| 0|Palsson, Master. ...| male|
 9| 1|Johnson, Mrs. Osc...|female|
 10| 1|Nasser, Mrs. Nich...|female|
 11| 1|Sandstrom, Miss. ...|female|
 12| 1|Bonnell, Miss. El...|female|
 13| 0|Saundercock, Mr. ...| male|
 14| 0|Andersson, Mr. An...| male|
 15| 0|Vestrom, Miss. Hu...|female|
 16| 1|Hewlett, Mrs. (Ma...|female|
 17| 0|Rice, Master. Eugene| male|
 18| 1|Williams, Mr. Cha...| male|
 19| 0|Vander Planke, Mr...|female|
 20| 1|Masselmani, Mrs. ...|female|
+-----------+--------+--------------------+------+
only showing top 20 rows

In [7]:
# Basic Analysis
# To get names of Columns 
ps_data_df.columns
tr_data_df.columns

# to get the schema 
ps_data_df.printSchema()
tr_data_df.printSchema()

# to get the number of rows in the dtaframe
ps_data_df.count() 
# it contains 891 observation
tr_data_df.count()  
# it contains 891 observation

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)

root
-- PassengerId: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Age: double (nullable = true)
-- Fare: double (nullable = true)

 Out[ 34 ]: 891

In [8]:
# Select Operations
print(ps_data_df.select(['PassengerId','Name']).show())
print(ps_data_df.selectExpr('Name as a ','  Survived  as b').show())

print(tr_data_df.select(['PassengerId','Pclass']).show())
print(tr_data_df.selectExpr('PassengerId as a ','  Pclass  as b').show())

+-----------+------+
PassengerId|Pclass|
+-----------+------+
 1| 3|
 2| 1|
 3| 3|
 4| 1|
 5| 3|
 6| 3|
 7| 1|
 8| 3|
 9| 3|
 10| 2|
 11| 3|
 12| 1|
 13| 3|
 14| 3|
 15| 3|
 16| 2|
 17| 3|
 18| 2|
 19| 3|
 20| 3|
+-----------+------+
only showing top 20 rows

None
+---+---+
 a| b|
+---+---+
 1| 3|
 2| 1|
 3| 3|
 4| 1|
 5| 3|
 6| 3|
 7| 1|
 8| 3|
 9| 3|
 10| 2|
 11| 3|
 12| 1|
 13| 3|
 14| 3|
 15| 3|
 16| 2|
 17| 3|
 18| 2|
 19| 3|
 20| 3|
+---+---+
only showing top 20 rows

None

In [9]:
# Filter Female Records
ps_data_df.filter(upper(col('Sex'))=='FEMALE').count()
#ps_data_df.filter(ps_data_df['Sex']=='female').count()
#ps_data_df.filter(ps_data_df.Sex=='female').count()
#ps_data_df.filter("Sex=='female'").count()

Out[ 66 ]: 314

In [10]:
# Create a new column named AgeTimeFare for tr_data_df
tr_data_df=tr_data_df.withColumn('AgeTimeFare',tr_data_df['Age']*tr_data_df['Fare'])

In [11]:
# Group by Pclass
grp_tr_dat_df=tr_data_df.groupBy('Pclass')
#grp_tr_dat_df.avg('Fare').show()
grp_tr_dat_df.agg({'PassengerId':'count','Age':'avg','Fare':'sum'}).show()

+------+------------------+------------------+------------------+
Pclass|count(PassengerId)| avg(Age)| sum(Fare)|
+------+------------------+------------------+------------------+
 1| 216|38.233440860215055|18177.412499999984|
 3| 491| 25.14061971830986| 6714.695100000002|
 2| 184| 29.87763005780347|3801.8416999999995|
+------+------------------+------------------+------------------+

In [12]:
# sORT THE dATAFrame
tr_data_df=tr_data_df.sort('Fare',ascending=True)

In [13]:
# JOin ps_data_df and tr_data_df 
# equi Join
maindf=ps_data_df.join(tr_data_df,on=['PassengerId'])
# Non Equi Join
non_equi_df=ps_data_df.join(tr_data_df,on=ps_data_df.PassengerId <= tr_data_df.PassengerId)

maindf.count()
non_equi_df.count()

Out[ 95 ]: 397386

In [14]:
# Union
ps_data_df.union(ps_data_df).sort('PassengerId',ascending=True).show()

+-----------+--------+--------------------+------+
PassengerId|Survived| Name| Sex|
+-----------+--------+--------------------+------+
 1| 0|Braund, Mr. Owen ...| male|
 1| 0|Braund, Mr. Owen ...| male|
 2| 1|Cumings, Mrs. Joh...|female|
 2| 1|Cumings, Mrs. Joh...|female|
 3| 1|Heikkinen, Miss. ...|female|
 3| 1|Heikkinen, Miss. ...|female|
 4| 1|Futrelle, Mrs. Ja...|female|
 4| 1|Futrelle, Mrs. Ja...|female|
 5| 0|Allen, Mr. Willia...| male|
 5| 0|Allen, Mr. Willia...| male|
 6| 0| Moran, Mr. James| male|
 6| 0| Moran, Mr. James| male|
 7| 0|McCarthy, Mr. Tim...| male|
 7| 0|McCarthy, Mr. Tim...| male|
 8| 0|Palsson, Master. ...| male|
 8| 0|Palsson, Master. ...| male|
 9| 1|Johnson, Mrs. Osc...|female|
 9| 1|Johnson, Mrs. Osc...|female|
 10| 1|Nasser, Mrs. Nich...|female|
 10| 1|Nasser, Mrs. Nich...|female|
+-----------+--------+--------------------+------+
only showing top 20 rows

Out[ 94 ]: 396940.5

In [16]:
tr_data_df.show()

+-----------+------+----+-------+
PassengerId|Pclass| Age| Fare|
+-----------+------+----+-------+
 1| 3| 22| 7.25|
 2| 1| 38|71.2833|
 3| 3| 26| 7.925|
 4| 1| 35| 53.1|
 5| 3| 35| 8.05|
 6| 3|null| 8.4583|
 7| 1| 54|51.8625|
 8| 3| 2| 21.075|
 9| 3| 27|11.1333|
 10| 2| 14|30.0708|
 11| 3| 4| 16.7|
 12| 1| 58| 26.55|
 13| 3| 20| 8.05|
 14| 3| 39| 31.275|
 15| 3| 14| 7.8542|
 16| 2| 55| 16|
 17| 3| 2| 29.125|
 18| 2|null| 13|
 19| 3| 31| 18|
 20| 3|null| 7.225|
+-----------+------+----+-------+
only showing top 20 rows

In [19]:
titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
PassengerId|Survived|Pclass| Name| Sex| Age|SibSp|Parch| Ticket| Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
 1| 0| 3|Braund, Mr. Owen ...| male| 22| 1| 0| A/5 21171| 7.25| null| S|
 2| 1| 1|Cumings, Mrs. Joh...|female| 38| 1| 0| PC 17599|71.2833| C85| C|
 3| 1| 3|Heikkinen, Miss. ...|female| 26| 0| 0|STON/O2. 3101282| 7.925| null| S|
 4| 1| 1|Futrelle, Mrs. Ja...|female| 35| 1| 0| 113803| 53.1| C123| S|
 5| 0| 3|Allen, Mr. Willia...| male| 35| 0| 0| 373450| 8.05| null| S|
 6| 0| 3| Moran, Mr. James| male|null| 0| 0| 330877| 8.4583| null| Q|
 7| 0| 1|McCarthy, Mr. Tim...| male| 54| 0| 0| 17463|51.8625| E46| S|
 8| 0| 3|Palsson, Master. ...| male| 2| 3| 1| 349909| 21.075| null| S|
 9| 1| 3|Johnson, Mrs. Osc...|female| 27| 0| 2| 347742|11.1333| null| S|
 10| 1| 2|Nasser, Mrs. Nich...|female| 14| 1| 0| 237736|30.0708| null| C|
 11| 1| 3|Sandstrom, Miss. ...|female| 4| 1| 1| PP 9549| 16.7| G6| S|
 12| 1| 1|Bonnell, Miss. El...|female| 58| 0| 0| 113783| 26.55| C103| S|
 13| 0| 3|Saundercock, Mr. ...| male| 20| 0| 0| A/5. 2151| 8.05| null| S|
 14| 0| 3|Andersson, Mr. An...| male| 39| 1| 5| 347082| 31.275| null| S|
 15| 0| 3|Vestrom, Miss. Hu...|female| 14| 0| 0| 350406| 7.8542| null| S|
 16| 1| 2|Hewlett, Mrs. (Ma...|female| 55| 0| 0| 248706| 16| null| S|
 17| 0| 3|Rice, Master. Eugene| male| 2| 4| 1| 382652| 29.125| null| Q|
 18| 1| 2|Williams, Mr. Cha...| male|null| 0| 0| 244373| 13| null| S|
 19| 0| 3|Vander Planke, Mr...|female| 31| 1| 0| 345763| 18| null| S|
 20| 1| 3|Masselmani, Mrs. ...|female|null| 0| 0| 2649| 7.225| null| C|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 20 rows